In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**INTRODUCCIÓN**

In [ ]:
# Daniel Revuelta González
# 03/05/2021
# Tarea 5

In [ ]:
# Importamos las librerías

import pandas as pd
import numpy as np

import gc
import time
import warnings

from scipy import sparse
import scipy.stats as ss

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image
import matplotlib_venn as venn

import string
import re
import nltk
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

import sys, os, re, csv, codecs
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
# Cargamos el dataset

path = '../input/'
comp = 'jigsaw-toxic-comment-classification-challenge/' # Conjunto de datos
EMBEDDING_FILE = f'{path}glove6b50d/glove.6B.50d.txt' # Incluimos Glove, que utilizaremos posteriormente
train_path = f'{path}{comp}train.csv.zip'
train = pd.read_csv(train_path)
test_path = f'{path}{comp}test.csv.zip'
test = pd.read_csv(test_path)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"] # Lista con las etiquetas


**ANÁLISIS EXPLORATORIO**

In [ ]:
train.tail(10) # Mostramos las 10 últimas observaciones del dataset de entreanmiento

In [ ]:
test.tail(10) # Mostramos las 10 últimas observaciones del dataset de testeo

In [ ]:
# Ejemplo de comentario "obsceno"

print("Comentario obsceno:")
print(train[train.obscene==1].iloc[1,1])


In [ ]:
# Ejemplo de comentario "tóxico"

print("Comentario tóxico:")
print(train[train.severe_toxic==1].iloc[3,1])

In [ ]:
# Analizamos si hay datos incompletos

print("Comprobación de valores perdidos en el conjunto de entrenamiento")
null_check=train.isnull().sum()
print(null_check)
print("Comprobación de valores perdidos en el conjunto de testeo")
null_check=test.isnull().sum()
print(null_check)

In [ ]:
# En este caso vemos que no hay ningún valor perdido, pero podemos sistematizar un par de sentencias que completen los datos si algún comentario no está disponible.

print("Completamos los datos no disponibles con \"_na_\"")
list_sentences_train = train["comment_text"].fillna("_na_").values
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [ ]:
# Marcamos las observaciones sin etiquetas como comentarios limpios

rowsums=train.iloc[:,2:].sum(axis=1) # Sumamos horizontalmente los valores de las etiquetas de cada comentario.
train['clean']=(rowsums==0) # Si la suma es 0, quiere decir que no tiene ninguna etiqueta asignada, por lo que se considera que es un comentario limpio. 

In [ ]:
# Total etiquetas

x=train.iloc[:,2:].sum() # Sumamos los valores de las etiquetas de todos los comentarios.
print("Total etiquetas =",x.sum())

# Comentarios totales

print("Total comentarios = ",len(train))

# Comentarios limpios

print("Total comentarios limpios = ",train['clean'].sum())

# Comentarios en los que se detecta toxicidad (pertenecen a alguna de las categorías citadas antes)

print("Total comentarios con presencia de toxicidad = ",len(train)-train['clean'].sum())

In [ ]:
# Se observa que el set de entrenamiento no está equilibrado, pues hay 143.346 comentarios limpios y 16.225 catalogados con otras etiquetas.
# Como vemos, hay más etiquetas que comentarios, por lo que se trata de un problema de clasificación "multi-clase" y "multi-etiquetas".

In [ ]:
# Graficamos según el tipo de etiqueta que tienen los comentarios:

plt.figure(figsize=(8,4))
ax= sns.barplot(x.index, x.values, alpha=0.8)
plt.title("Apariciones según la clase de comentario")
plt.ylabel('Nº de apariciones', fontsize=12)
plt.xlabel('Tipo de etiqueta ', fontsize=12)

# Añadimos las etiquetas
rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
plt.show()

In [ ]:
# En el gráfico se aprecia cómo el dataset se encuentra desequilibrado, siendo la etiqueta "clean" la mayoritaria. Porcentualmente, la distribución sería:

for z in list_classes:
    print('La clase',z, 'supone un: ',100*train[z].sum()/len(train), "% del total")
print('La clase clean supone un: ',100*train["clean"].sum()/len(train), "% del total")

In [ ]:
# Graficamos según el número de etiquetas

x=rowsums.value_counts()

plt.figure(figsize=(8,4))
ax = sns.barplot(x.index, x.values, alpha=0.8)
plt.title("Cantidad de etiquetas por comentario")
plt.ylabel('Número de comentarios', fontsize=12)
plt.xlabel('Número de etiquetas ', fontsize=12)

# Añadimos las etiquetas
rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
plt.show()

In [ ]:
# Hay 143.346 comentarios sin etiquetas (son limpios)
# Hay 6.360 comentarios que pertenecen a una única categoría, 4.209 a dos, 3.480 a 3, 1.760 a 4 y 385 a 5.
# Se aprecia que incluso hay 31 comentarios que están catalogados en todos los tipos de comentarios tóxicos.

In [ ]:
# Comprobamos la relación entre etiquetas (en este caso, "toxic" contra el resto)
main_col="toxic"

# Aunque en esta ocasión se ha efectuado únicamente para la etiqueta "toxic" para reducir el tiempo de ejecución, se puede hacer con otras, pues sería similar el proceso. 

temp_df=train.iloc[:,2:-1] # Quitamos los comentarios limpios porque no tienen etiqueta

# Al ser variables categóricas, vamos a efectuarlo mediante tablas de contingencia

corr_mats=[]
for other_col in temp_df.columns[1:]:
    confusion_matrix = pd.crosstab(temp_df[main_col], temp_df[other_col])
    corr_mats.append(confusion_matrix)
out = pd.concat(corr_mats,axis=1,keys=temp_df.columns[1:])

print('Matriz: ')
print(out)

# También se puede hacer con una matriz de confusión y el estadístico de Cramer: #https://stackoverflow.com/questions/20892799/using-pandas-calculate-cram%C3%A9rs-coefficient-matrix/39266194


In [ ]:
# Graficamos un WordCloud con las palabras más mencionadas en comentarios "limpios"

subset=train[train.clean==True]
text=subset.comment_text.values
wc= WordCloud(background_color="black",max_words=2000).generate(" ".join(text)) # Se fija un máximo de 2000 palabras
plt.title("Palabras más utilizadas en comentarios limpios", fontsize=12)
plt.imshow(wc,interpolation='none')
plt.axis("off")
plt.show()

In [ ]:
# Podemos hacer lo mismo con comentarios clasificados como "obscenos"

subset=train[train.obscene==True]
text=subset.comment_text.values
wc= WordCloud(background_color="black",max_words=2000).generate(" ".join(text)) # Se fija un máximo de 2000 palabras
plt.title("Palabras más utilizadas en comentarios obscenos", fontsize=12)
plt.imshow(wc,interpolation='none')
plt.axis("off")
plt.show()

**DESARROLLO DEL MODELO**

In [ ]:
# Se fija su configuración básica

embed_size = 50 # Cómo de grande va a ser el vector con el texto
max_features = 20000 # Cuántas palabras únicas se incorporan al vector
maxlen = 100 # Máximo número de palabras a usar en un comentario


In [ ]:
# Se inicia el tokenizador, con el número máximo de palabras que hemos establecido (20000)
tokenizer = Tokenizer(num_words=max_features)

# Creamos el diccionario de índices y palabras del vocabulario según el orden de aparición de estas en los comentarios que se analizan.
# Nota: el 0 se reserva para el padding.
tokenizer.fit_on_texts(list(list_sentences_train)) 

# Transformamos los comentarios en una secuencia de números, sustituyendo cada palabra por el índice que ocupa en el diccionario.
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

# Convertimos las secuencias de números obtenidas en el paso anterior en cadena de igual longitud (en este caso, 100)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
# Leemos los vectores de palabras de Glove:

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [ ]:
# Se utilizan los vectores para crear la matriz de embedding, con inicio alatorio para las palabras que no se encuentran en Glove. 
# Para esa inicialización aleatoria utilizaremos la media y la desviación estándar de los embedding de Glove.

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

In [ ]:
word_index = tokenizer.word_index # Diccionario con las palabras y su número asignado tras realizar el ajuste sobre el texto con el tokenizador (fit_on_texts) 
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size)) # Matriz de embedding
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
# Se construye el modelo:

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp) 
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Entrenamos el modelo:

model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1);

In [ ]:
# Predecimos las variables de las etiquetas:

y_test = model.predict([X_te], batch_size=1024, verbose=1)

In [ ]:
# Mostramos las 10 primeras predicciones:

y_test[:10,]

In [ ]:
# Planteamos un modelo alternativo, al que se ha añadido otra capa densa para ver si ello mejora el desempeño del modelo.

# Añadimos un Early Stopping como buena práctica, para que se interrumpa el entrenamiento si la función de pérdida sobre el conjunto de validación empeora durante 2 epochs:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2)

# Además, como otra buena práctica, representaremos gráficamente la evolución de las distintas funciones de pérdida y de precisión durante el proceso de entreanmiento para monitorizar qué ocurre

In [ ]:
# Se construye el modelo:

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Ajustamos nuestro modelo alternativo:

model_2 = model.fit(X_t, y, batch_size=32, epochs=5, callbacks=[callbacks], validation_split=0.1);

In [ ]:
# Seleccionamos los datos de la función de pérdida y la precisión del set de entrenamiento y el de validación

model_2_loss = model_2.history['loss']
model_2_val_loss = model_2.history['val_loss']
model_2_accuracy = model_2.history['accuracy']
model_2_val_accuracy = model_2.history['val_accuracy']


# Determinamos el número de epochs realizados durante el proceso de entrenamiento
num_epochs = range(1,len(model_2_loss)+1)

# Graficamos la función de pérdida
plt.plot(num_epochs,model_2_loss,label='Training_Loss')
plt.plot(num_epochs,model_2_val_loss,label='Validation_Loss')
plt.title('Pérdida en el entrenamiento y en la validación')
plt.xlabel('Nº epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Graficamos la precisión
plt.plot(num_epochs,model_2_accuracy,label='Training_Acc')
plt.plot(num_epochs,model_2_val_accuracy,label='Validation_Acc')
plt.title('Precisión en el entrenamiento y en la validación')
plt.xlabel('Nº epoch')
plt.ylabel('Precisión')
plt.legend()
plt.show()


In [ ]:
# Predecimos las etiquetas del conjunto de testeo:

y_test_2 = model.predict([X_te], batch_size=1024, verbose=1)

In [ ]:
# Mostramos las 10 primeras predicciones:

y_test_2[:10,]